### Import modules

In [8]:
import pandas as pd
import io
import os,sys
import urllib.request, urllib.parse, urllib.error
import pandas_datareader.data as pdr
import datetime
import numpy as np
from tqdm import tqdm
import pathlib


### Create a python function to access CSV data from Intrinio, and put it into a pandas DataFrame

In [ ]:
def get_from_intrinio(stk,start_date=None,end_date=None):
    sd = '2016-01-01' if start_date is None else start_date
    ed = str(datetime.datetime.now())[0:10]
    try:
        url = f'https://api.intrinio.com/prices.csv?start_date={sd}&end_date={ed}&identifier={stk}&api_key=OmU1NmEzYTU1Yjg3N2NjYWRiOWFjM2Y2OGYzMWNlMDcx'
        req = urllib.request.Request(url)
        f = urllib.request.urlopen(req)
    except Exception:
        f = urllib.request.urlopen(req)
    alines = f.read().decode('utf-8')        
    csv = alines.split('\n')[1:]
    handle = io.StringIO('\n'.join(csv))
    df = pd.read_csv(handle)
    return df


### Get Data

In [ ]:
sym = 'AAPL'
df1 = get_from_intrinio(sym)

In [ ]:
df1.head()

In [ ]:
df1.tail()

### Show it's done using yahoo

In [ ]:
def fetch_history(symbol,dt_beg,dt_end):
    df = pdr.DataReader(symbol, 'yahoo', dt_beg, dt_end)
    df['Date'] = df.index
    df.Date = df.Date.astype(str).str.slice(0,10)
    df.columns = df.columns.get_level_values(0)
    df.index = list(range(len(df)))
    return df


### Analyis of gap up's and downs

In [ ]:
for sym in tqdm(list(df_comp3.Symbol.values)):
    time.sleep(.01)

In [ ]:
exch = 'nyse'
home_dir = str(pathlib.Path.home())
df_comp = pd.read_csv(f'{home}/downloads/{exch}_companylist.csv')
df_comp.columns = [c.strip() for c in df_comp.columns.values]
num_cols = ['MarketCap','LastSale']
for c in num_cols:
    df_comp[c] = df_comp[c].astype(str).str.replace(',','').astype(float)
df_comp2 = df_comp[(df_comp.MarketCap>100000000) & (df_comp.MarketCap<2000000000)]
df_comp2 = df_comp2[df_comp2.LastSale<=30]
print(f'len(df_comp2): {len(df_comp2)}')
df_comp3 = df_comp2[['Symbol']+num_cols].sort_values('MarketCap')
df_comp3.tail(20)
dict_df_smallcap = {}
sym_list = list(df_comp3.Symbol.values)
for sym in tqdm(sym_list):
    print(f'fetching {sym}')
    try:
        df_temp = fetch_history(sym,'2016-01-01','2019-06-14')
        dict_df_smallcap[sym] = df_temp
    except:
        print(f'no data for {sym}')
print(f'total rows: {np.array([len(dict_df_smallcap[k]) for k in dict_df_smallcap.keys()]).sum()}')
dict_large_pchg = {}
for k in dict_df_smallcap:
    dft = dict_df_smallcap[k]
    dft['Symbol'] = k
    dft['prev_close'] = dft.Close.shift(1)
    dft['pchg'] = dft.Close.pct_change()
    dft2 = dft[dft.pchg.abs()>.49]
    if len(dft2)>0:
        dict_large_pchg[k] = dft2
print(f'len of large pchgs: {len(dict_large_pchg)}')
# print(dict_large_pchg[list(dict_large_pchg.keys())[0]])
df_large_chg = dict_large_pchg[list(dict_large_pchg.keys())[0]]
for k in list(dict_large_pchg.keys())[1:]:
    df_large_chg = df_large_chg.append(dict_large_pchg[k])
    df_large_chg.index = list(range(len(df_large_chg)))
df_large_chg = df_large_chg.sort_values(['Symbol','Date']) 
df_large_chg.to_csv(f'{home_dir}/downloads/df_large_chg_{exch}.csv',index=False)
